In [1]:
import time
import RPi.GPIO as GPIO
from time import sleep
from picamera2 import Picamera2, Preview
from picamera2.controls import Controls
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile
from datetime import datetime
import tkinter as tk

In [2]:
picam2 = Picamera2()
led = 23
GPIO.setmode(GPIO.BCM)
GPIO.setup(led, GPIO.OUT)

[3:14:43.177051130] [7103]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70
[3:14:43.231443154] [7118]  INFO RPI raspberrypi.cpp:1476 Registered camera /base/soc/i2c0mux/i2c@1/imx477@1a to Unicam device /dev/media1 and ISP device /dev/media3
/tmp/ipykernel_7103/1460731898.py:4: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(led, GPIO.OUT)


In [3]:
# Set camera controls
controls = {"ExposureTime": 4000000, #microseconds
            "AnalogueGain":1.0, # 1 = ISO 100
            "AeEnable": False, # Auto exposure and Gain
            "AwbEnable": False,# Auto white Balance
            "FrameDurationLimits": (114,239000000)} #Min/Max frame duration

# Setup config parameters
preview_config = picam2.create_preview_configuration(raw={"size": picam2.sensor_resolution, "format": "SBGGR12",},
                                                     controls = controls) 

In [6]:
def capture_raw(suffix):
    picam2.start_preview(Preview.QTGL)
    picam2.configure(preview_config)

    picam2.start() # Start Camera
 
    time.sleep(2)

    GPIO.output(led, GPIO.HIGH) # Turn on LED
    time.sleep(1)

    #Capture image in unpacked RAW format 12bit dynamic range (16bit array)
    raw = picam2.capture_array("raw").view(np.uint16)

    time.sleep(1)
    GPIO.output(led, GPIO.LOW) # Turn off LED

    print(picam2.capture_metadata())

    picam2.stop_preview()
    picam2.stop()

    raw_crop = raw[0:3040, 0:4056] # Remove padding from each row of pixels

    base_filename = "RAW"
    save_dir = '/home/martin/Desktop/Images/'

    # Create a new folder with date stamp if it does not exist
    date_str = datetime.now().strftime("%Y-%m-%d")
    save_dir = os.path.join(save_dir, f'{base_filename}_{date_str}')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Construct the filename with the user-defined suffix
    #filename = f'{base_filename}_{date_str}_air_sat_{suffix}.tiff'

    # Check if the filename with the user-defined suffix already exists in the folder
    filename = f'{base_filename}_{date_str}_air_sat{suffix}.tiff'
    count = 1
    while os.path.exists(os.path.join(save_dir, filename)):
        # If the filename exists, add a number to the suffix and try again
        filename = f'{base_filename}_{date_str}_air_sat{suffix}_{count}.tiff'
        count += 1

    # Save the image with the updated filename
    tifffile.imwrite(os.path.join(save_dir, filename), raw_crop)
    print("RAW Image Saved")

#Create a function to get the value of the suffix from the entry field and call the capture_raw function
def capture_image():
    suffix_value = suffix_entry.get()
    capture_raw(suffix_value)

# Create the Tkinter window and add the button and entry field
window = tk.Tk()
window.title("Capture RAW Image")
window.geometry("300x200")
suffix_label = tk.Label(window, text="Enter Suffix Value:")
suffix_label.pack()
suffix_entry = tk.Entry(window)
suffix_entry.pack()
capture_button = tk.Button(window, text="Capture Raw Image", command=capture_image)
capture_button.pack()
window.mainloop()

[3:21:01.006261643] [7103]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-XBGR8888 (1) 4056x3040-SBGGR12
[3:21:01.023031918] [7118]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 4056x3040-SBGGR12_1X12 - Selected unicam format: 4056x3040-BG12


{'FocusFoM': 3716, 'ExposureTime': 3999942, 'ColourTemperature': 4000, 'SensorTimestamp': 12069718935000, 'ScalerCrop': (2, 0, 4053, 3040), 'SensorBlackLevels': (4096, 4096, 4096, 4096), 'DigitalGain': 1.000014305114746, 'ColourGains': (2.510904550552368, 1.656680941581726), 'SensorTemperature': 23.0, 'Lux': 1.8981562852859497, 'FrameDuration': 4000571, 'AeLocked': True, 'ColourCorrectionMatrix': (2.1575498580932617, -1.1039637327194214, -0.05359601974487305, -0.3457588851451874, 1.6122934818267822, -0.2665446400642395, -0.02813815139234066, -0.7009111046791077, 1.7290493249893188), 'AnalogueGain': 1.0}
RAW Image Saved


[3:21:18.028469935] [7103]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-XBGR8888 (1) 4056x3040-SBGGR12
[3:21:18.045183414] [7118]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 4056x3040-SBGGR12_1X12 - Selected unicam format: 4056x3040-BG12


{'FocusFoM': 3600, 'ExposureTime': 3999942, 'ColourTemperature': 4000, 'SensorTimestamp': 12086720698000, 'ScalerCrop': (2, 0, 4053, 3040), 'SensorBlackLevels': (4096, 4096, 4096, 4096), 'DigitalGain': 1.000014305114746, 'ColourGains': (2.510904550552368, 1.656680941581726), 'SensorTemperature': 23.0, 'Lux': 1.8079651594161987, 'FrameDuration': 4000571, 'AeLocked': True, 'ColourCorrectionMatrix': (2.1575498580932617, -1.1039637327194214, -0.05359601974487305, -0.3457588851451874, 1.6122934818267822, -0.2665446400642395, -0.02813815139234066, -0.7009111046791077, 1.7290493249893188), 'AnalogueGain': 1.0}
RAW Image Saved


[3:21:34.711229281] [7103]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-XBGR8888 (1) 4056x3040-SBGGR12
[3:21:34.727883926] [7118]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 4056x3040-SBGGR12_1X12 - Selected unicam format: 4056x3040-BG12


{'FocusFoM': 3730, 'ExposureTime': 3999942, 'ColourTemperature': 4000, 'SensorTimestamp': 12103422716000, 'ScalerCrop': (2, 0, 4053, 3040), 'SensorBlackLevels': (4096, 4096, 4096, 4096), 'DigitalGain': 1.000014305114746, 'ColourGains': (2.510904550552368, 1.656680941581726), 'SensorTemperature': 23.0, 'Lux': 1.8977383375167847, 'FrameDuration': 4000571, 'AeLocked': True, 'ColourCorrectionMatrix': (2.1575498580932617, -1.1039637327194214, -0.05359601974487305, -0.3457588851451874, 1.6122934818267822, -0.2665446400642395, -0.02813815139234066, -0.7009111046791077, 1.7290493249893188), 'AnalogueGain': 1.0}
RAW Image Saved


[3:21:47.422837794] [7103]  INFO Camera camera.cpp:1028 configuring streams: (0) 640x480-XBGR8888 (1) 4056x3040-SBGGR12
[3:21:47.439624254] [7118]  INFO RPI raspberrypi.cpp:851 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 4056x3040-SBGGR12_1X12 - Selected unicam format: 4056x3040-BG12


{'FocusFoM': 3659, 'ExposureTime': 3999942, 'ColourTemperature': 4000, 'SensorTimestamp': 12116156246000, 'ScalerCrop': (2, 0, 4053, 3040), 'SensorBlackLevels': (4096, 4096, 4096, 4096), 'DigitalGain': 1.000014305114746, 'ColourGains': (2.510904550552368, 1.656680941581726), 'SensorTemperature': 24.0, 'Lux': 1.8682752847671509, 'FrameDuration': 4000571, 'AeLocked': True, 'ColourCorrectionMatrix': (2.1575498580932617, -1.1039637327194214, -0.05359601974487305, -0.3457588851451874, 1.6122934818267822, -0.2665446400642395, -0.02813815139234066, -0.7009111046791077, 1.7290493249893188), 'AnalogueGain': 1.0}
RAW Image Saved


In [ ]:
center_crop = raw[1000:2000, 2000:3000]

#Get color channels in bayer order (BGGR)
red = center_crop[1::2,1::2]
green1 = center_crop[0::2,1::2]
green2 = center_crop[1::2,0::2]
green = np.add(green1,green2)/2

#Make histogram for red and green channel # Set camera controls to have good pixel saturation
Colors=("red","green")
Channel_ids=(red,green)
for channel_id, c in zip(Channel_ids,Colors):
    histogram, bin_edges=np.histogram(channel_id,bins=4095, range=(0,4095))
    plt.plot(bin_edges[0:-1],histogram,color=c)
plt.title("Red_Green histogram")
plt.xlabel("Pixel intensity")
plt.ylabel("Pixel Frequency")
plt.show()
plt.imshow(raw_crop, cmap="gray")
